# Using OpenFisca US

This notebook shows how to use OpenFisca US to compute the impact of reforms to tax-benefit policies modelled.

## 1. Simulating the baseline

In [1]:
from openfisca_us import IndividualSim, Microsimulation

# OpenFisca US has two main classes that can be instantiated.
# IndividualSim() creates an object representing a single household,
# allowing you to manually specify income, demographics and expenses.

# Microsimulation() creates an object representing a survey dataset of
# households, allowing you to simulate country-wide impacts without
# having to manually specify households.

sim = IndividualSim() # We'll specify a household.
sim.add_person(age=26, name="adult", employment_income=30_000) # Add people, specifying their input variables.
sim.add_tax_unit(members=["adult"]) # Add groups, specifying the inputs and members.
sim.calc("income_tax") # sim.calc (in both classes) takes a variable and simulates its formula.


array([1840.5], dtype=float32)

In [2]:
sim = Microsimulation()
sim.calc("income_tax") # Here, sim.calc returns a much larger array.

             value       weight
0      -560.000000   687.710022
1         0.000000   687.710022
2      5289.000000  1516.949951
3      1240.500000  1516.949951
4         0.000000  1483.560059
...            ...          ...
84167     0.000000   470.359985
84168  5276.950195   312.350006
84169     0.000000   514.109985
84170  9041.113281   516.250000
84171  3512.040039   386.369995

[84172 rows x 2 columns]

## 2. Defining a reform

In [6]:
from openfisca_us.model_api import * # The model API gives us all the tools to specify reforms.

sim = IndividualSim()
sim.add_person(age=26, name="adult", employment_income=30_000)
sim.add_tax_unit(members=["adult"])

class reform(Reform):
    def apply(self):
        self.neutralize_variable("standard_deduction") # Force the standard deduction to zero.
        # self.modify_parameters(f: parameters -> parameters) updates the parameter tree
        # self.update_variable(new variable class definition) changes a variable.
        # self.add_variable(new variable class definition) adds a variable.

reformed_sim = IndividualSim(reform)
reformed_sim.add_person(age=26, name="adult", employment_income=30_000)
reformed_sim.add_tax_unit(members=["adult"])

reformed_tax = reformed_sim.calc("income_tax")[0]
baseline_tax = sim.calc("income_tax")[0]

print(f"Income tax increases by ${reformed_tax - baseline_tax:,.0f}")

Income tax increases by $1,554
